# Projeto: Engenharia de Dados e Garantia de Qualidade no Conjunto de Dados do Airbnb no Rio de Janeiro

In [ ]:
#instalar adaptador
pip install psycopg2


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   --------- ------------------------------ 0.3/1.2 MB ? eta -:--:--
   ------------------ --------------------- 0.5/1.2 MB 1.9 MB/s eta 0:00:01
   --------------------------- ------------ 0.8/1.2 MB 1.5 MB/s eta 0:00:01
   ------------------------------------ --- 1.0/1.2 MB 1.2 MB/s eta 0:00:01
   ---------------------------------------- 1.2/1.2 MB 1.1 MB/s eta 0:00:00


In [58]:
#importar bibliotecas
import pandas as pd
import psycopg2
import numpy as np

In [37]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

### Camada Bronze
A camada bronze salva no banco de dados do PostgreSQL contém os dados em seu formato original

In [70]:
#estabelecer conexão com a camada bronze
conn = psycopg2.connect(
    host = 'localhost', 
    database ="Bronze_Layer", 
    user ="postgres",  
    password = 131415  
)

In [71]:
# criar cursor
cur_review = conn.cursor()
#selecionar conteúdo da tabela
cur_review.execute('SELECT * FROM reviews_table_bronze')
#selecionar todas as linhas
rows_review = cur_review.fetchall()
#transformar em dataframe
reviews = pd.DataFrame(rows_review, columns=['listing_id', 'id', 'date', 'reviewer_id', 'reviewer_name', 'comments'])

In [72]:
cur_listings = conn.cursor()
cur_listings.execute('SELECT * FROM listings_table_bronze')
rows_listings = cur_listings.fetchall()
listings = pd.DataFrame(rows_listings, columns=['id', 'listing_url', 'scrape_id', 'last_scraped', 'source', 'name', 'description', 'neighborhood_overview', 'picture_url', 'host_id', 
'host_url', 'host_name', 'host_since', 'host_location', 'host_about', 'host_response_time', 'host_response_rate', 'host_acceptance_rate', 
'host_is_superhost', 'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood', 'host_listings_count', 'host_total_listings_count', 
'host_verifications', 'host_has_profile_pic', 'host_identity_verified', 'neighbourhood', 'neighbourhood_cleansed', 
'neighbourhood_group_cleansed', 'latitude', 'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms', 'bathrooms_text', 
'bedrooms', 'beds', 'amenities', 'price', 'minimum_nights', 'maximum_nights', 'minimum_minimum_nights', 'maximum_minimum_nights', 
'minimum_maximum_nights', 'maximum_maximum_nights', 'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'calendar_updated', 
'has_availability', 'availability_30', 'availability_60', 'availability_90', 'availability_365', 'calendar_last_scraped', 
'number_of_reviews', 'number_of_reviews_ltm', 'number_of_reviews_l30d', 'first_review', 'last_review', 'review_scores_rating', 
'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 
'review_scores_value', 'license', 'instant_bookable', 'calculated_host_listings_count', 'calculated_host_listings_count_entire_homes', 
'calculated_host_listings_count_private_rooms', 'calculated_host_listings_count_shared_rooms', 'reviews_per_month'
])

In [73]:
cur_calendar = conn.cursor()
cur_calendar.execute('SELECT * FROM calendar_table_bronze')
rows_calendar = cur_calendar.fetchall()
calendar = pd.DataFrame(rows_calendar, columns=['listing_id', 'date', 'avaliable', 'price', 'adjusted_price', 'minimum_nights', 'maximum_nights'])

### Camada Silver
A camada Silver contém dados padronizados, limpos e métricas de qualidade implementadas

Data Clean - Camada Silver: <br>
- Identifique e lide com valores ausentes, duplicatas e outliers nos dados brutos da camada "bronze". <br>
- Padronize e limpe os nomes das colunas, convertendo-os em um formato consistente. <br>
- Realize uma limpeza textual em campos, como descrições de propriedades, removendo caracteres especiais e erros de digitação.

In [74]:
#converter reviews para os tipos corretos
reviews[['listing_id', 'id', 'reviewer_id']] = reviews[['listing_id', 'id', 'reviewer_id']].apply(pd.to_numeric)
reviews['date'] = pd.to_datetime(reviews['date'])

In [75]:
#converter calendar para os tipos corretos e remover caracteres especiais
calendar['price'] = calendar['price'].str.replace('$','', regex=False)
calendar['price'] = calendar['price'].str.replace(',','', regex=False).astype(float)
calendar['date'] = pd.to_datetime(calendar['date'])
calendar[['listing_id', 'adjusted_price', 'minimum_nights', 'maximum_nights']] = calendar[['listing_id', 'adjusted_price', 'minimum_nights', 'maximum_nights']].apply(pd.to_numeric)

In [76]:
#converter listings para os tipos corretos e remover caracteres especiais
converter_numerico = [
    'id', 'host_id', 'host_listings_count', 'host_total_listings_count', 
    'accommodates', 'bathrooms', 'bedrooms', 'beds', 'minimum_nights', 
    'maximum_nights', 'minimum_minimum_nights', 'maximum_minimum_nights', 
    'minimum_maximum_nights', 'maximum_maximum_nights', 'minimum_nights_avg_ntm', 
    'maximum_nights_avg_ntm', 'availability_30', 'availability_60', 
    'availability_90', 'availability_365', 'number_of_reviews', 
    'number_of_reviews_ltm', 'number_of_reviews_l30d', 'review_scores_rating', 
    'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 
    'review_scores_communication', 'review_scores_location', 'review_scores_value', 
    'calculated_host_listings_count', 'calculated_host_listings_count_entire_homes', 
    'calculated_host_listings_count_private_rooms', 
    'calculated_host_listings_count_shared_rooms', 'reviews_per_month'
]

converter_data = [
    'last_scraped', 'host_since', 'first_review', 'last_review'
]

#converter os tipos conforme as listas acima
listings[converter_numerico] = listings[converter_numerico].apply(pd.to_numeric)
listings[converter_data] = listings[converter_data].apply(pd.to_datetime)

#corrigir a coluna de price que tinha caractere especial
listings['price'] = listings['price'].str.replace('$','', regex=False)
listings['price'] = listings['price'].str.replace(',','', regex=False).astype(float)

#existem valores N/A nas colunas host_response_rate e host_acceptance_rate; é prevciso alterar para NaN
listings['host_response_rate'] = listings['host_response_rate'].replace('N/A', np.NaN)
listings['host_acceptance_rate'] =listings['host_acceptance_rate'].replace('N/A', np.NaN)

#converter colunas com % para formato adequado
listings['host_response_rate'] = listings['host_response_rate'].str.replace('%', '', regex=False).astype(float) / 100
listings['host_acceptance_rate'] = listings['host_acceptance_rate'].str.replace('%', '', regex=False).astype(float) / 100

In [ ]:
#algumas colunas de listings estão com vazio 'None', vou mudar para NaN
contem_vazios = ['description', 'neighborhood_overview', 'host_location', 'host_response_rate', 
'host_acceptance_rate', 'host_is_superhost', 'host_neighbourhood', 'neighbourhood', 
'neighbourhood_group_cleansed', 'bathrooms', 'bathrooms_text', 'bedrooms', 
'beds', 'price', 'calendar_updated', 'has_availability', 'first_review', 
'last_review', 'review_scores_rating', 'review_scores_accuracy', 
'review_scores_cleanliness', 'review_scores_checkin', 
'review_scores_communication', 'review_scores_location', 
'review_scores_value', 'license', 'reviews_per_month']

listings[contem_vazios] = listings[contem_vazios].fillna(np.nan)

In [ ]:
# para concluir essa etapa ainda aprecisa:
#tratar outros caracteres especiais
#remover erros de digitação
#verificar se tem linhas duplicadas nas bases
#eu não entendi essa parte: Padronize e limpe os nomes das colunas, convertendo-os em um formato consistente. > creio que as colunas já estejam padronizadas
#verifiquei as linhas vazias nas bases; depois que eu dei uma mexida, acho que tudo que está vazio está como NaN corretamente, agora eu não sei se cabe aos engenheiros 'tratar' isso 

Data Quality - Camada Silver:
- Defina métricas de qualidade de dados, como integridade, precisão e consistência para os dados da camada "bronze".
- Implemente verificações para garantir que os dados da camada "silver" estejam em conformidade com essas métricas.
- Estabeleça um sistema de monitoramento contínuo da qualidade dos dados da camada "silver".